# Jupyter dashboard app for RC visualizations

In [1]:
# Imports
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import dash
from dash import Dash, dcc, html, callback, callback_context
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import dash_daq as daq
from dash.exceptions import PreventUpdate
from dash import dash_table
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import geopandas as gpd
from shapely.geometry import Polygon
from os import path
from os import remove
from os import listdir
import re
import warnings
import sys
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor as RFR
import base64
import io

## Pre-processing for RC app
All of he funtions used in Dash app

In [2]:
warnings.filterwarnings('ignore')

def read_data(variables, DF_RC):
    """ READ_DATA

        Objective: Read the dataframes and variables to consider and return prdictors and dependent variable seperated
        Input: 
            - Variables: List with names of the variables to be considered in analysis
            - DF_RC: Dataframe with dependent and independent variables.
        Output:
            - X: Predictors (Ind. vars.)
            - y: Predicted varaiable (Dep. var.)
            - msg: Message with information of the process.
    """ 
    
    X = DF_RC[variables]
    y = DF_RC['RC']

    X = sm.add_constant(X)
    
    msg = 'Reading data...'
    
    return X, y, msg

def fit_model(X, y, HQ, model = 'Log_Linear'):
    """ FIT_MODEL

        Objective: Take X, y and the type of model and fits the data to the given model.
        
        Input: 
            - X: Predictors (Ind. vars.)
            - y: Predicted varaiable (Dep. var.)
            - HQ: Boolean. True if a High Quality result is wanted. False if HQ is not required.
            - model: Model to be fitted (Options are Log_linear or Random_Forest)
        Output:
            - importance: Either a table with weights and p-values estimated (Log_linear) or feature importances (Random Forest)
            - RMSE: Root Mean Squared Error calculated using the Leav On Out Crooss-Validation Approach.
            - mod: Model fitted
            - exists: Boolean that says if file aleady has been saved.
            - name_file: Name of he file to be saved if it doesn't exist.
            - msg: Message with information of the process.
    """ 
    variables = list(X.columns)[1:]
    
    var_ID = ''
    for i in variables:
        var_ID += i[0]
    
    if model == 'Log_Linear':
        lin_reg = sm.OLS(np.log(y), X).fit(maxiter=1000)
        mod = lin_reg
        importance = lin_reg.params
        pval = lin_reg.pvalues
        importance = pd.DataFrame(importance[1:])
        importance['Features'] = variables
        importance['p'] = pval
        importance.columns = ['Weight', 'Features','p_value']
        importance = importance[['Features', 'Weight', 'p_value']]
        importance['Weight'] = importance.Weight.apply(lambda df : round(df,2))
        importance['p_value'] = importance.p_value.apply(lambda df : round(df,2))
        
        # CROSS VALIDATION SCORE 
        df = pd.concat([X, np.log(y)], axis = 1)
        errors = []

        # LOOCV
        for i in range(len(df)):

            # splitting our dataset into train and test datasets.
            train = pd.concat([df.iloc[:i], df.iloc[(i+1):]])
            test = pd.DataFrame(df.iloc[i]).T
            lin_reg = sm.OLS(train.RC, train.iloc[:,:-1]).fit(maxiter=1000000)

            RMSE = np.sqrt(np.sum((np.exp(lin_reg.predict(test.iloc[:,:-1])) - np.exp(test.RC))**2)/1)
            errors.append(RMSE)
        
        RMSE = np.mean(errors)
        RMSE = pd.DataFrame([np.round(RMSE,2)])
        RMSE.columns = ['RMSE']
        
    elif model == 'Random_Forest': 
        RF_reg = RFR().fit(X, y)
        mod = RF_reg
        importance = RF_reg.feature_importances_
        importance = pd.DataFrame(importance[1:])
        importance['Features'] = variables
        importance.columns = ['Importance', 'Features']
        importance = importance[['Features', 'Importance']]
        importance['Importance'] = importance.Importance.apply(lambda df : round(df,2))
        
        # CROSS VALIDATION SCORE 
        df = pd.concat([X, y], axis = 1)
        errors = []

        # LOOCV
        for i in range(len(df)):

            # splitting our dataset into train and test datasets.
            train = pd.concat([df.iloc[:i, 1:], df.iloc[(i+1):, 1:]])
            test = pd.DataFrame(df.iloc[i]).T
            RF = RFR().fit(train.iloc[:,:-1], train.RC)

            RMSE = np.sqrt(np.sum((RF.predict(test.iloc[:,1:-1]) - test.RC)**2)/1)
            errors.append(RMSE)
        
        RMSE = np.mean(errors)
        RMSE = pd.DataFrame([RMSE])
        RMSE.columns = ['RMSE']
        
    importance = importance.to_dict('records')
    
    RMSE = RMSE.to_dict('records')
    
    name_file = model+var_ID
    
    if HQ:
        exists = path.exists('Regression results/Rn_estimations_'+name_file+'_pol_HQ.geojson')
    else:
        exists = path.exists('Regression results/Rn_estimations_'+name_file+'_pol.geojson')
        
    if exists:
        msg = 'Fitting model...\nEstimating RC values...'
    else:
        msg = 'Fitting model...\nEstimating RC values...\n\nThis will take some time...\n\n'
    
        
    return importance, RMSE, mod, exists, name_file, msg


def EstimatingValues(mod,
                     model,
                     X,
                     exists,
                     HQ,
                     name_file,
                     df_RnModel,
                     crs = '3116',
                     res = 300,
                     ):
    """ EstimatingValues

        Objective: Apply fitted model to the household dataset.
        
        Input: 
            - mod: Model fitted
            - model: Model to be fitted (Options are Log_linear or Random_Forest)
            - X: Predictors (Ind. vars.)
            - exists: Boolean that says if file aleady has been saved.
            - HQ: Boolean. True if a High Quality result is wanted. False if HQ is not required.
            - name_file: Name of he file to be saved if it doesn't exist.
            - df_RnModel: Dataset to which the model is applied.
            - crs. Coordinate system EPSG (string)
            - res: Spatial resolution in meters.
        Output:
            - grid: Grid with the results of predicted Rn values.
            - x_c: centroid X coordinate
            - y_c: centroid Y coordinate
            - msg: Message with information of the process.
    """ 
    if exists:
        
        if HQ:
            grid = gpd.read_file('Regression results/Rn_estimations_'+name_file+'_pol_HQ.geojson')
        else:
            grid = gpd.read_file('Regression results/Rn_estimations_'+name_file+'_pol.geojson')
    else:
        df_RnModel['const'] = np.ones(len(df_RnModel))

        x_range = df_RnModel['X'].max() - df_RnModel['X'].min()
        y_range = df_RnModel['Y'].max() - df_RnModel['Y'].min()

        df_RnModel['Cluster'] = np.zeros(len(df_RnModel))

        cols = np.arange(df_RnModel['X'].min() + res/2, df_RnModel['X'].max(), res)
        rows = np.arange(df_RnModel['Y'].min() + res/2, df_RnModel['Y'].max(), res)

        k = 0
        polygons = []
        for i in range(len(cols)-1):

            for j in range(len(rows)-1):

                k += 1
                df_RnModel.loc[(df_RnModel.X >= cols[i])&(df_RnModel.X < cols[i+1])&(df_RnModel['Y'] >= rows[j])&(df_RnModel['Y'] < rows[j+1]), 'Cluster'] = k
                polygons.append(Polygon([(cols[i],rows[j]), (cols[i]+res, rows[j]), (cols[i]+res, rows[j]+res), (cols[i], rows[j]+res)]))    

        df_RnModel = df_RnModel.groupby('Cluster').mean()

        df_RnModel_reg = df_RnModel[list(X.columns)]
        
        if model == 'Log_Linear':
            df_RnModel['RC'] = np.exp(mod.predict(df_RnModel_reg))
        elif model == 'Random_Forest':
            df_RnModel['RC'] = mod.predict(df_RnModel_reg)
            
        gdf = gpd.GeoDataFrame(df_RnModel['RC'], geometry=gpd.points_from_xy(df_RnModel.X, df_RnModel.Y))
        gdf = gdf.set_crs('EPSG:'+crs)
        gdf = gdf.to_crs('EPSG:4326')
        grid = gpd.GeoDataFrame({'geometry':polygons})
        grid = grid.set_crs('EPSG:'+crs)
        grid = grid.to_crs('EPSG:4326')
        grid = gpd.sjoin(grid, gdf)
        
        if HQ:
            # df_RnModel.to_csv('Regression results/Rn_estimations_'+name_file+'_HQ.csv')
            # gdf.to_file('Regression results/Rn_estimations_'+name_file+'_HQ.geojson', driver="GeoJSON")
            grid.to_file('Regression results/Rn_estimations_'+name_file+'_pol_HQ.geojson', driver="GeoJSON")
        else:
            # df_RnModel.to_csv('Regression results/Rn_estimations_'+name_file+'.csv')
            # gdf.to_file('Regression results/Rn_estimations_'+name_file+'.geojson', driver="GeoJSON")
            grid.to_file('Regression results/Rn_estimations_'+name_file+'_pol.geojson', driver="GeoJSON")
    
    msg = 'Done :)'
    
    x_c = grid.dissolve().centroid.x.mean()
    y_c = grid.dissolve().centroid.y.mean()

    return grid, x_c, y_c, msg

def plot_figure(Organization, DF_RC):
    """ plotFigure

        Objective: Plot radon concentration distribution and pie plot with percentage of concentrations above some reference level
        
        Input: 
            - Organization: Which reference level to compare it with (WHO or EPA)
            - DF_RC: Dataframe with dependent and independent variables.
        Output:
            - fig: Plotly figure with Radon distribution and pie plot.
            - DF_data: Dataset with a column of exceedence
    """ 
    DF_data = pd.DataFrame(DF_RC['RC'])

    DF_data['Exceed WHO'] = DF_data.RC.apply(lambda df :'Above WHO recommended level' if  (df >100) else 'Below WHO recmmended level')
    DF_data['Exceed EPA'] = DF_data.RC.apply(lambda df :'Above EPA action level' if  (df >148) else 'Below EPA action level')
    
    RC_max = np.max(DF_data['RC'])
    
    x = np.arange(0,(RC_max - RC_max%25) + 26,25)
    rc = DF_data['RC']
    y = np.histogram(rc, bins = x)

    hist = px.histogram(DF_data, x = 'RC', range_x = [0,(RC_max - RC_max%25) + 25])

    fig=make_subplots(specs=[[{'secondary_y': True}, {"type": "pie"}]],
                      cols = 2)

    fig.update_layout(template = 'morph',
                     );

    fig.add_trace(
        go.Histogram(x=hist.data[0].x,
               y=hist.data[0].y,
               name="Percentage of<br>RC measurements", 
               histnorm = 'percent', marker_color = 'rgb(55,100,200)',
               hoverinfo = 'x+y',
              ), secondary_y=False)
    
    ref_levs = [100,148]
    
    if Organization[-3:] == 'WHO':
        fig.add_vline(ref_levs[0], annotation_text = Organization[-3:] + ' recommended level',
                      annotation_position = 'top',
                      line_dash="dash", row = 1, col =1)
    else:
        fig.add_vline(ref_levs[1], annotation_text = Organization[-3:] + ' recommended level',
                      annotation_position = 'top',
                      line_dash="dash", row = 1, col =1)

    fig.update_traces(xbins=dict( # bins used for histogram
            start=0.0,
            end=425.0,
            size=25
            ))
    
    fig.update_xaxes(range = [0, max(rc) + max(rc)*0.05])

    fig.add_trace(
        go.Scatter(x = (x[1:]),
                   y = np.round(100*np.cumsum(y[0]/30),2),
                   name="Accumulated<br>percentage of<br>RC measurements",
                   line_color="#ee0000", hoverinfo="x+y"), secondary_y=True)

    fig.update_layout(title_text = 'Residential RC measurements distribution', 
                      title_font_family = 'bahnschrift',
                      font_family = 'bahnschrift',
                      title_font_size = 30, xaxis_title_text='Residential RC [Bq/m^3]', # xaxis label
                      yaxis_title_text='Percentage of RC measurements' # yaxis label
                     )

    labels = DF_data.groupby(Organization).count().iloc[:,0].index
    values = DF_data.groupby(Organization).count().iloc[:,0].values

    fig.add_trace(go.Pie(labels = labels,
                         values = values,
                         textinfo = 'percent',
                         hoverinfo = 'label+value', 
                         marker = dict(colors = ['rgb(255,0,0)', 'rgb(55,100,200)']),
                         showlegend = False,
                         title = 'Comparison with ' + Organization[-3:] + ' recommedation',
                         titleposition = 'bottom center',
                         titlefont = dict(size = 20)
                        ),
                  row = 1, col = 2 
                 )


    fig.update_layout(title_font_size = 30)
    
    return fig, DF_data

# App 
Create and run the dash app

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MORPH])

load_figure_template(["morph"])
color = 'lightsteelblue'

printing = html.P(["printing", html.Br()])

app.layout = html.Div([html.Div([html.Div([],style = {'width':20}), html.H1('Residential RC modeling', style={'font-family' : 'bahnschrift'})],  style = {'display':'flex'}),
                       html.Div([''], style = {'height':20, 'background-color':color}),
                       html.Div([html.H5('Upload here your datasets before you continue',
                                         style={'font-family' : 'bahnschrift'}),
                                 html.Div([html.Div([dcc.Upload(id='upload-data-f',
                                                                children=html.Div(['Drag and Drop or ',
                                                                                   html.B('Select Files for fitting the model (measurements)')
                                                                                  ]),
                                                                style={'width': '935px',
                                                                       'height': '60px',
                                                                       'color' : 'darkblue',
                                                                       'lineHeight': '60px',
                                                                       'borderWidth': '1px',
                                                                       'borderStyle': 'dashed',
                                                                       'borderRadius': '5px',
                                                                       'textAlign': 'center',
                                                                       'margin': '10px'
                                                                      }
                                                               )
                                                    ]),
                                           html.Div([dcc.Upload(id='upload-data-r',
                                                                children=html.Div(['Drag and Drop or ',
                                                                                   html.B('Select Files for applying the model (Cadaster information)')
                                                                                  ]),
                                                                style={'width': '935px',
                                                                       'height': '60px',
                                                                       'color' : 'darkred',
                                                                       'lineHeight': '60px',
                                                                       'borderWidth': '1px',
                                                                       'borderStyle': 'dashed',
                                                                       'borderRadius': '5px',
                                                                       'textAlign': 'center',
                                                                       'margin': '10px'
                                                                      }
                                                               )
                                                    ]),
                                          ],
                                          style = dict(display='flex',  width =1900)
                                         ),
                                 html.Div([''], style = {'height':20, 'background-color':color}),
                                 html.Div([html.Div([''], style = {'width':20}),
                                           html.Div([html.H5('Compare measurements with recommendation levels', 
                                                             style={'font-family' : 'bahnschrift'}),
                                                     dcc.Dropdown(['Exceed WHO','Exceed EPA'],
                                                                  'Exceed WHO',
                                                                  id='Organization',
                                                                  style={'font-family' : 'bahnschrift','width':440}
                                                                 ),
                                                     html.H5('Feature and model selection for RC modeling', style={'font-family' : 'bahnschrift'}),
                                                     html.Div([html.Plaintext('   Feature selection information: ',
                                                                              style={'font-family' : 'bahnschrift', 'width' : 250}
                                                                             ),
                                                               dcc.Dropdown(['Correlation matrix', 'Variance Inflation Factor'],
                                                                            'Correlation matrix',
                                                                            id = 'FS', style={'font-family' : 'bahnschrift', 'width' : 200}
                                                                           )
                                                              ],
                                                              style=dict(display='flex', width = 440)
                                                             ),
                                                     html.Div([html.Plaintext('   Model: ',
                                                                              style={'font-family' : 'bahnschrift', 'width' : 100}
                                                                             ),
                                                               dcc.Dropdown(['Log_Linear', 'Random_Forest'],
                                                                            'Log_Linear',
                                                                            id = 'model',
                                                                            style={'font-family' : 'bahnschrift', 'width' : 340}
                                                                           )
                                                              ], 
                                                              style=dict(display='flex', width = 440)
                                                             ),
                                                     html.Div([html.Plaintext('   Features: ',
                                                                              style={'font-family' : 'bahnschrift', 'width' : 100}
                                                                             ),
                                                               dcc.Dropdown([],
                                                                            False,
                                                                            id = 'vars_',
                                                                            style={'font-family' : 'bahnschrift' , 'width' : 340},
                                                                            multi = True
                                                                           )
                                                              ], 
                                                              style=dict(display='flex')
                                                             ),
                                                     html.Div([], style = {'height': 10}),
                                                     html.Div([html.Plaintext('   High quality model:',
                                                                              style={'font-family' : 'bahnschrift', 'width' : 170}
                                                                             ),
                                                               html.Div([html.Div([''], style = {'height':15}),
                                                                         daq.BooleanSwitch(id='HQ_model', on=False),
                                                                        ]
                                                                       ),
                                                              ],
                                                              style=dict(display='flex', width = 440)),
                                                     html.Div([html.Div([html.Plaintext('   Run model: ',
                                                                                        style={'font-family' : 'bahnschrift', 'width' : 100}
                                                                                       ),
                                                                         html.Button('RUN',
                                                                                     style={'font-family' : 'bahnschrift',
                                                                                            'background-color':'steelblue',
                                                                                            'font-size':'20px',
                                                                                            'border' : '0px',
                                                                                            'color': 'white',
                                                                                            'border-radius':'12px',
                                                                                            'width' : 340,
                                                                                            'height' :50},
                                                                                     id='Predict_Rn',
                                                                                     n_clicks=0),
                                                                        ],
                                                                        style=dict(display='flex', width = 465)
                                                                       ),
                                                              ],
                                                              style=dict(display='flex', width = 450)),
                                                    ],
                                                    style = {'width' : 450}
                                                   ), 
                                           html.Div([''], style = {'width':20, 'background-color':color}),
                                           html.Div([dcc.Graph(id='RC-histogram',
                                                               style = {'height' : 470, 'width' : 995}
                                                              ),
                                                     html.Div([''], style = {'width':20, 'background-color':color}),
                                                     dcc.Graph(id = 'FS_out',
                                                               style = {'height' : 470, 'width' : 400}
                                                              )
                                                    ],
                                                    style=dict(display='flex'))
                                          ],
                                          style=dict(display='flex')),
                                 html.Div([''],style = {'height':20, 'background-color':color}),
                                 html.Div([html.Div([''], style = {'width':20}),
                                           html.Div([html.H5(' Regression results: ',
                                                             style={'font-family' : 'bahnschrift'}
                                                            ),
                                                     dash_table.DataTable(id= 'imp',
                                                                          style_table={'width' : 440}
                                                                         ),
                                                     html.H6(' Cross-validation score:', style={'font-family' : 'bahnschrift'}),
                                                     dash_table.DataTable(id= 'RMSE',
                                                                          style_table={'width' : 440}
                                                                         )
                                                    ]
                                                   ),
                                           html.Div([''],style={'width':10}),
                                           html.Div([''],style={'width':20, 'background-color':color}),
                                           dcc.Graph(id='RC-model-map',
                                                     style = {'height' : 540, 'width' : 1460},
                                                     config = {'displayModeBar': False})
                                          ],
                                          style=dict(display='flex')
                                         ),
                                 html.Div([''], style = {'height':20,'background-color':color}),
                                 html.Div([''], style = {'height':20}),
                                 html.Div([html.Div([html.P('Dashboard created by:'),
                                                     html.A('Martín Domínguez Durán',
                                                            href='https://www.martindominguezduran.com',
                                                            target="_blank")
                                                    ],
                                                    style = {'width':1460}),
                                           html.Div([html.Plaintext('   Reset modeling environment: ',
                                                                    style={'font-family' : 'bahnschrift'}
                                                                   ),
                                                     html.Button('RESET',
                                                                 style={'font-family' : 'bahnschrift',
                                                                        'background-color':'darkred',
                                                                        'font-size':'14px',
                                                                        'border' : '0px',
                                                                        'color': 'white',
                                                                        'border-radius':'12px',
                                                                        'width' : 100,
                                                                        'height' :60},
                                                                 id='RestartModel',
                                                                 n_clicks=0)
                                                    ],
                                                    style=dict(display='flex',width = 400))
                                          ],
                                          style=dict(display='flex',width = 1900)
                                         ),
                                 html.P(id = 'none'),
                                 
                                ]
                               ),
                      ])
                                          
                                

@app.callback(
    [Output('RC-histogram', 'figure'), Output('vars_', 'options')],
    [Input('Organization', 'value'), Input('upload-data-f','contents')])
def update_graph(Organization, list_of_contents):
    
    if list_of_contents is not None:
        
        cont = list_of_contents.split(',')[1]
        
        decoded = base64.b64decode(cont)
    
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        
        fig, df_ = plot_figure(Organization, df)
        
        var_names = list(df.iloc[:,2:].columns)
        
    else:
        fig = make_subplots(cols = 1)

        fig.update_layout(template = 'morph',
                          yaxis=dict(showgrid = False, tickvals = [], zeroline = False),
                          xaxis=dict(showgrid = False, tickvals = [], zeroline = False)
                         )
        
        var_names = []
    
    return fig, var_names

lst_clicks_rstrt = []

@app.callback(
    Output('none', 'children'),
    Input('RestartModel','n_clicks')
)
def Restart(RestartModel):
    
    lst_clicks_rstrt.append(RestartModel)
    
    if len(lst_clicks_rstrt) == 1:
        raise PreventUpdate
    elif lst_clicks_rstrt[-1] > lst_clicks_rstrt[-2]:
        for f in listdir('Regression results'):
            if re.search('^RC_regression_estimations', f):
                remove(path.join('Regression results', f))
            if re.search('^Rn_estimations', f):
                remove(path.join('Regression results', f))
                
    return ''


@app.callback(
    Output('FS_out', 'figure'),
    [Input('FS','value'), Input('upload-data-f','contents')]
)
def feature_sel(info_FS, list_of_contents):
    if list_of_contents is not None:
        
        cont = list_of_contents.split(',')[1]
        
        decoded = base64.b64decode(cont)
    
        DF_RC = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        
        if info_FS == 'Correlation matrix':
            cor = DF_RC.corr().iloc[1:,1:]

            for i in range(len(cor)):
                for j in range(len(cor)):
                    if i < j:
                        cor.iloc[i,j] = np.nan


            fig = px.imshow(cor, color_continuous_scale='RdBu_r', zmin = -1, zmax = 1)

            fig.update_traces(hoverinfo = 'z', hovertemplate = "r_pearson: %{z:.2f}")

        elif info_FS == 'Variance Inflation Factor':
            
            X = DF_RC.iloc[:,2:]
            X = sm.add_constant(X)

            tab = pd.DataFrame()
            tab["Features"] = X.columns[1:]
            tab["VIF Factor"] = [round(variance_inflation_factor(X.values, i+1),2) for i in range(X.shape[1]-1)]
            VIF_vars = list(tab[tab['VIF Factor'] < 4]['Features'])
            tab = tab.sort_values(by = 'VIF Factor', ascending = False)
            tab=tab.set_index('Features')
            
            fig = px.imshow(tab, color_continuous_scale='RdBu_r', zmax = 4, zmin = 0)
            fig.update_coloraxes(showscale=False)
            fig.update_traces(hoverinfo = 'z', hovertemplate = "VIF: %{z:.2f}")

        fig.update_layout(title_text = 'Information for feature selection', 
                          title_font_family = 'bahnschrift',
                          font_family = 'bahnschrift')
    else:
        
        fig = make_subplots(cols = 1)

        fig.update_layout(template = 'morph',
                          yaxis=dict(showgrid = False, tickvals = [], zeroline = False),
                          xaxis=dict(showgrid = False, tickvals = [], zeroline = False)
                         )
            
    return fig
    

lst_clicks_mp = []

@app.callback(
    [Output('RC-model-map', 'figure'),Output('imp', 'data'),Output('RMSE', 'data')],
    [Input('Predict_Rn','n_clicks'),Input('model', 'value'), Input('vars_','value'), Input('HQ_model','on'), Input('upload-data-f', 'contents'), Input('upload-data-r', 'contents')]
)

def update_map(Predict_Rn, model, vars_, HQ, DF_RC_c, df_RnModel_c):
    
    lst_clicks_mp.append(Predict_Rn)
    
    if ((Predict_Rn == 0)&(len(lst_clicks_mp) == 2)):
        df = pd.DataFrame([[0,-72]])
        df_ = pd.DataFrame(['  '])
        df_.columns = [' ']
        imp = df_.to_dict('records')
        RMSE = imp
        fig = px.scatter_mapbox(df, lat = 0, lon = 1, opacity = 0)
        
        fig.update_traces(hoverinfo = 'skip', hovertemplate = " ")
        fig.update_layout(mapbox_style="carto-positron",
                          mapbox_zoom = 1.5)
        
    elif (lst_clicks_mp[-1] == lst_clicks_mp[-2]):
        raise PreventUpdate
        
    elif (lst_clicks_mp[-1] > lst_clicks_mp[-2]):
        if DF_RC_c is not None:
        
            cont = DF_RC_c.split(',')[1]

            decoded = base64.b64decode(cont)

            DF_RC = pd.read_csv(io.StringIO(decoded.decode('utf-8')))

        if df_RnModel_c is not None:

            cont = df_RnModel_c.split(',')[1]

            decoded = base64.b64decode(cont)

            df_RnModel = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        
        if  HQ:    
            # print('HQ')
            X, y, msg = read_data(vars_, DF_RC)
            print(msg)
            imp, RMSE, mod, exists, name_file, msg = fit_model(X,y,HQ, model = model)
            print(msg)
            rc_pol, x_c, y_c, msg = EstimatingValues(mod, model, X, exists,HQ, name_file, df_RnModel, res=100)
            print(msg)
        else:
            # print('LQ')
            X, y, msg = read_data(vars_, DF_RC)
            print(msg)
            imp, RMSE, mod, exists, name_file, msg = fit_model(X, y, HQ, model = model)
            print(msg)
            rc_pol, x_c, y_c, msg = EstimatingValues(mod, model, X, exists, HQ, name_file, df_RnModel)
            print(msg)

        fig = px.choropleth_mapbox(rc_pol,
                                    geojson=rc_pol.geometry,
                                    locations = rc_pol.index,
                                    color='RC',
                                    color_continuous_scale="Portland",
                                    opacity = 0.65,
                                    hover_data= ['RC']
                                   )
        
        fig.update_traces(marker_line_width = 0, hoverinfo = 'z')

        fig.update_layout(mapbox_style="carto-positron",
                          mapbox_center = {'lat':y_c, 'lon':x_c},
                          mapbox_zoom = 10)
    
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        
    return fig, imp, RMSE

if __name__ == '__main__':
    app.run_server(debug=True, port = 8070)
    

Dash app running on http://127.0.0.1:8070/
